# Evolution PDE Stability

In [ ]:
import taichi as ti
ti.init(arch=ti.cpu, debug=False)
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
from tqdm import tqdm
# %matplotlib widget

In [ ]:
from dsfilter.SE2.LI.derivatives import(
    laplacian,
    morphological
)
from dsfilter.SE2.LI.filter import (
    compute_timestep_diffusion,
    compute_timestep_shock
)
import dsfilter

## Diffusion

In [ ]:
test_case = 1

In [ ]:
match test_case:
    case 1: # Grid of lines (black)
        dim_I, dim_J = 128, 128
        u = np.ones((dim_I, dim_J)) * 255.
        N_lines = 4
        offset = dim_I // (N_lines + 1)
        for k in range(N_lines):
            centre = (k + 1) * offset
            u[(centre-1):(centre+2), :] = 0.
            u[:, (centre-1):(centre+2)] = 0.
        xs, ys = np.meshgrid(np.linspace(-1, 1, dim_I), np.linspace(-1, 1, dim_J))
        mask = (xs**2 + ys**2) < 0.6
        u[mask] = 255.

        G_D_inv = np.array((1., 1., 0.05))

        u = sp.ndimage.gaussian_filter(u, 2)
    case 2: # Grid of lines (alternating black and white)
        dim_I, dim_J = 128, 128
        u = np.ones((dim_I, dim_J)) * 0.5 * 255.
        N_lines = 5
        offset = dim_I // (N_lines + 1)
        colour = 0. # black
        for k in range(N_lines):
            centre = (k + 1) * offset
            u[(centre-1):(centre+2), :] = colour
            u[:, (centre-1):(centre+2)] = 255. - colour
            colour = 255. - colour
        xs, ys = np.meshgrid(np.linspace(-1, 1, dim_I), np.linspace(-1, 1, dim_J))
        mask = (xs**2 + ys**2) < 0.6
        u[mask] = 0.5 * 255.

        G_D_inv = np.array((1., 1., 0.05))

        u = sp.ndimage.gaussian_filter(u, 2)


dim_I, dim_J = u.shape
dim_K = 16
Is, Js, Ks = np.indices((dim_I, dim_J, dim_K))
x_min, x_max = 0., dim_I - 1.
y_min, y_max = 0., dim_J - 1.
θ_min, θ_max = 0., 2 * np.pi
dxy = (x_max - x_min) / (dim_I - 1)
dθ = (θ_max - θ_min) / dim_K
xs, ys, θs = dsfilter.SE2.utils.coordinate_array_to_real(Is, Js, Ks, x_min, y_min, θ_min, dxy, dθ)

In [ ]:
cws = dsfilter.orientationscore.cakewavelet_stack(dim_I, dim_K, Gaussian_σ=dim_I/8)
U = dsfilter.orientationscore.wavelet_transform(u, cws.real)
U = np.transpose(U, axes=(1, 2, 0)) # x, y, θ

In [ ]:
K = 0
print(θs[0, 0, K])
fig, ax = plt.subplots(1, 2, figsize=(12, 5))
_, _, cbar = dsfilter.visualisations.plot_image_array(U[..., K], x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[0])
ax[0].set_title(f"$U(\cdot, {θs[0, 0, K]:.2f})$")
fig.colorbar(cbar, ax=ax[0])
_, _, cbar = dsfilter.visualisations.plot_image_array(U.sum(-1), x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[1])
ax[1].set_title("$\int_θ U(\cdot, θ) dθ$")
fig.colorbar(cbar, ax=ax[1]);

In [ ]:
def diffusion(U_np, G_inv_np, dxy, dθ, θs_np, T):
    U = ti.field(dtype=ti.f32, shape=U_np.shape)
    U.from_numpy(U_np)
    G_inv = ti.Matrix(G_inv_np, dt=ti.f32)
    θs = ti.field(dtype=ti.f32, shape=θs_np.shape)
    θs.from_numpy(θs_np)
    laplacian_u = ti.field(dtype=ti.f32, shape=U.shape)
    τ = compute_timestep_diffusion(dxy, dθ, G_D_inv)
    print(τ)
    for _ in tqdm(range(T)):
        laplacian(U, G_inv, dxy, dθ, θs, laplacian_u)
        step_diffusion(U, τ, laplacian_u)
    return U.to_numpy()
    

@ti.kernel
def step_diffusion(
    U: ti.template(),
    τ: ti.f32,
    laplacian_u: ti.template()
):
    for I in ti.grouped(laplacian_u):
        U[I] += τ * laplacian_u[I]

In [ ]:
U_diffused_isotropic = diffusion(U, np.array((1., 1., 0.001)), dxy, dθ, θs, 1000)
U_diffused_anisotropic = diffusion(U, np.array((1., 0., 0.001)), dxy, dθ, θs, 1000)

In [ ]:
print(U.min(), U.max())
print(U_diffused_isotropic.min(), U_diffused_isotropic.max())
print(U_diffused_anisotropic.min(), U_diffused_anisotropic.max())

In [ ]:
K = 0
print(θs[0, 0, K])
fig, ax = plt.subplots(1, 2, figsize=(12, 5))
_, _, cbar = dsfilter.visualisations.plot_image_array(U_diffused_anisotropic[..., K], x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[0])
ax[0].set_title(f"$U(\cdot, {θs[0, 0, K]:.2f})$")
fig.colorbar(cbar, ax=ax[0])
_, _, cbar = dsfilter.visualisations.plot_image_array(U_diffused_anisotropic.sum(-1), x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[1])
ax[1].set_title("$\int_θ U(\cdot, θ) dθ$")
fig.colorbar(cbar, ax=ax[1]);

In [ ]:
K = 0
print(θs[0, 0, K])
fig, ax = plt.subplots(1, 2, figsize=(12, 5))
_, _, cbar = dsfilter.visualisations.plot_image_array(U_diffused_isotropic[..., K], x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[0])
ax[0].set_title(f"$U(\cdot, {θs[0, 0, K]:.2f})$")
fig.colorbar(cbar, ax=ax[0])
_, _, cbar = dsfilter.visualisations.plot_image_array(U_diffused_isotropic.sum(-1), x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[1])
ax[1].set_title("$\int_θ U(\cdot, θ) dθ$")
fig.colorbar(cbar, ax=ax[1]);

In [ ]:
np.max(np.abs(U_diffused_anisotropic - U_diffused_isotropic))

In [ ]:
K = 0
print(θs[0, 0, K])
fig, ax = plt.subplots(1, 2, figsize=(12, 5))
_, _, cbar = dsfilter.visualisations.plot_image_array((U_diffused_anisotropic - U_diffused_isotropic)[..., K], x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[0])
ax[0].set_title(f"$U(\cdot, {θs[0, 0, K]:.2f})$")
fig.colorbar(cbar, ax=ax[0])
_, _, cbar = dsfilter.visualisations.plot_image_array((U_diffused_anisotropic - U_diffused_isotropic).sum(-1), x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[1])
ax[1].set_title("$\int_θ U(\cdot, θ) dθ$")
fig.colorbar(cbar, ax=ax[1]);

## Shock

### Dilation & Erosion

In [ ]:
test_case = 3

In [ ]:
match test_case:
    case 1: # Grid of lines (black)
        dim_I, dim_J = 128, 128
        u = np.ones((dim_I, dim_J)) * 255.
        N_lines = 4
        offset = dim_I // (N_lines + 1)
        for k in range(N_lines):
            centre = (k + 1) * offset
            u[(centre-1):(centre+2), :] = 0.
            u[:, (centre-1):(centre+2)] = 0.
        xs, ys = np.meshgrid(np.linspace(-1, 1, dim_I), np.linspace(-1, 1, dim_J))
        mask = (xs**2 + ys**2) < 0.6
        u[mask] = 255.

        G_D_inv = np.array((1., 1., 0.05))

        u = sp.ndimage.gaussian_filter(u, 2)
    case 2: # Grid of lines (alternating black and white)
        dim_I, dim_J = 128, 128
        u = np.ones((dim_I, dim_J)) * 0.5 * 255.
        N_lines = 5
        offset = dim_I // (N_lines + 1)
        colour = 0. # black
        for k in range(N_lines):
            centre = (k + 1) * offset
            u[(centre-1):(centre+2), :] = colour
            u[:, (centre-1):(centre+2)] = 255. - colour
            colour = 255. - colour
        xs, ys = np.meshgrid(np.linspace(-1, 1, dim_I), np.linspace(-1, 1, dim_J))
        mask = (xs**2 + ys**2) < 0.6
        u[mask] = 0.5 * 255.

        G_D_inv = np.array((1., 1., 0.05))

        u = sp.ndimage.gaussian_filter(u, 2)


    case 3: # Grid of lines (alternating black and white)
        dim_I, dim_J = 128, 128
        u = np.ones((dim_I, dim_J))
        u[:dim_I//2, :] = 0.

        G_D_inv = np.array((1., 1., 0.05))

        u = sp.ndimage.gaussian_filter(u, 2)


dim_I, dim_J = u.shape
dim_K = 16
Is, Js, Ks = np.indices((dim_I, dim_J, dim_K))
x_min, x_max = 0., dim_I - 1.
y_min, y_max = 0., dim_J - 1.
θ_min, θ_max = 0., 2 * np.pi
dxy = (x_max - x_min) / (dim_I - 1)
dθ = (θ_max - θ_min) / dim_K
xs, ys, θs = dsfilter.SE2.utils.coordinate_array_to_real(Is, Js, Ks, x_min, y_min, θ_min, dxy, dθ)

In [ ]:
cws = dsfilter.orientationscore.cakewavelet_stack(dim_I, dim_K, Gaussian_σ=dim_I/8)
U = dsfilter.orientationscore.wavelet_transform(u, cws.real)
U = np.transpose(U, axes=(1, 2, 0)) # x, y, θ

In [ ]:
U = np.ones((dim_I, dim_J, dim_K))
U[:dim_I//2] = 0.

In [ ]:
K = 0
print(θs[0, 0, K])
fig, ax = plt.subplots(1, 2, figsize=(12, 5))
_, _, cbar = dsfilter.visualisations.plot_image_array(U[..., K], x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[0])
ax[0].set_title(f"$U(\cdot, {θs[0, 0, K]:.2f})$")
fig.colorbar(cbar, ax=ax[0])
_, _, cbar = dsfilter.visualisations.plot_image_array(U.sum(-1), x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[1])
ax[1].set_title("$\int_θ U(\cdot, θ) dθ$")
fig.colorbar(cbar, ax=ax[1]);

In [ ]:
def morph(U_np, G_inv_np, dxy, dθ, θs_np, T):
    U_dilation = ti.field(dtype=ti.f32, shape=U_np.shape)
    U_dilation.from_numpy(U_np)
    U_erosion = ti.field(dtype=ti.f32, shape=U_np.shape)
    U_erosion.from_numpy(U_np)
    G_inv = ti.Matrix(G_inv_np, dt=ti.f32)
    θs = ti.field(dtype=ti.f32, shape=θs_np.shape)
    θs.from_numpy(θs_np)
    dilation_u = ti.field(dtype=ti.f32, shape=U.shape)
    erosion_u = ti.field(dtype=ti.f32, shape=U.shape)
    τ = compute_timestep_shock(dxy, dθ, G_inv)
    print(τ)
    for _ in tqdm(range(T)):
        morphological(U_dilation, G_inv, dxy, dθ, θs, dilation_u, erosion_u)
        step_dilation(U_dilation, τ, dilation_u)
        step_erosion(U_erosion, τ, erosion_u)
    return U_dilation.to_numpy(), U_erosion.to_numpy(), erosion_u.to_numpy(), dilation_u.to_numpy()
    
@ti.kernel
def step_dilation(
    U: ti.template(),
    τ: ti.f32,
    dilation_u: ti.template()
):
    for I in ti.grouped(dilation_u):
        U[I] += τ * dilation_u[I]
        
@ti.kernel
def step_erosion(
    U: ti.template(),
    τ: ti.f32,
    erosion_u: ti.template()
):
    for I in ti.grouped(erosion_u):
        U[I] += τ * erosion_u[I]

In [ ]:
U_dilation_isotropic, U_erosion_isotropic, erosion_isotropic, dilation_isotropic = morph(U, np.array((1., 1., 0.001)), dxy, dθ, θs, 100)
U_dilation_anisotropic, U_erosion_anisotropic, erosion_anisotropic, dilation_anisotropic = morph(U, np.array((1., 0., 0.)), dxy, dθ, θs, 100)

In [ ]:
print(U.min(), U.max())
print(U_dilation_isotropic.min(), U_dilation_isotropic.max())
print(U_dilation_anisotropic.min(), U_dilation_anisotropic.max())
print(U_erosion_isotropic.min(), U_erosion_isotropic.max())
print(U_erosion_anisotropic.min(), U_erosion_anisotropic.max())

In [ ]:
K = 0
print(θs[0, 0, K])
fig, ax = plt.subplots(2, 2, figsize=(12, 10))
_, _, cbar = dsfilter.visualisations.plot_image_array((erosion_anisotropic)[..., K], x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[0, 0])
ax[0, 0].set_title(f"$U(\cdot, {θs[0, 0, K]:.2f})$")
fig.colorbar(cbar, ax=ax[0, 0])
_, _, cbar = dsfilter.visualisations.plot_image_array((erosion_anisotropic).sum(-1), x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[0, 1])
ax[0, 1].set_title("$\int_θ U(\cdot, θ) dθ$")
fig.colorbar(cbar, ax=ax[0, 1])
_, _, cbar = dsfilter.visualisations.plot_image_array(dilation_anisotropic[..., K], x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[1, 0])
ax[1, 0].set_title(f"$U(\cdot, {θs[0, 0, K]:.2f})$")
fig.colorbar(cbar, ax=ax[1, 0])
_, _, cbar = dsfilter.visualisations.plot_image_array(dilation_anisotropic.sum(-1), x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[1, 1])
ax[1, 1].set_title("$\int_θ U(\cdot, θ) dθ$")
fig.colorbar(cbar, ax=ax[1, 1]);

In [ ]:
K = 0
print(θs[0, 0, K])
fig, ax = plt.subplots(2, 2, figsize=(12, 10))
_, _, cbar = dsfilter.visualisations.plot_image_array((U_dilation_anisotropic)[..., K], x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[0, 0])
ax[0, 0].set_title(f"$U(\cdot, {θs[0, 0, K]:.2f})$")
fig.colorbar(cbar, ax=ax[0, 0])
_, _, cbar = dsfilter.visualisations.plot_image_array(U_dilation_anisotropic.sum(-1), x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[0, 1])
ax[0, 1].set_title("$\int_θ U(\cdot, θ) dθ$")
fig.colorbar(cbar, ax=ax[0, 1])
_, _, cbar = dsfilter.visualisations.plot_image_array((U_erosion_anisotropic)[..., K], x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[1, 0])
ax[1, 0].set_title(f"$U(\cdot, {θs[0, 0, K]:.2f})$")
fig.colorbar(cbar, ax=ax[1, 0])
_, _, cbar = dsfilter.visualisations.plot_image_array(U_erosion_anisotropic.sum(-1), x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[1, 1])
ax[1, 1].set_title("$\int_θ U(\cdot, θ) dθ$")
fig.colorbar(cbar, ax=ax[1, 1]);

In [ ]:
K = 0
print(θs[0, 0, K])
fig, ax = plt.subplots(2, 2, figsize=(12, 10))
_, _, cbar = dsfilter.visualisations.plot_image_array(U_dilation_isotropic[..., K], x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[0, 0])
ax[0, 0].set_title(f"$U(\cdot, {θs[0, 0, K]:.2f})$")
fig.colorbar(cbar, ax=ax[0, 0])
_, _, cbar = dsfilter.visualisations.plot_image_array(U_dilation_isotropic.sum(-1), x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[0, 1])
ax[0, 1].set_title("$\int_θ U(\cdot, θ) dθ$")
fig.colorbar(cbar, ax=ax[0, 1])
_, _, cbar = dsfilter.visualisations.plot_image_array(U_erosion_isotropic[..., K], x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[1, 0])
ax[1, 0].set_title(f"$U(\cdot, {θs[0, 0, K]:.2f})$")
fig.colorbar(cbar, ax=ax[1, 0])
_, _, cbar = dsfilter.visualisations.plot_image_array(U_erosion_isotropic.sum(-1), x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[1, 1])
ax[1, 1].set_title("$\int_θ U(\cdot, θ) dθ$")
fig.colorbar(cbar, ax=ax[1, 1]);

In [ ]:
print(np.max(np.abs(U_dilation_anisotropic - U_dilation_isotropic)))
print(np.max(np.abs(U_erosion_anisotropic - U_erosion_isotropic)))

In [ ]:
K = 0
print(θs[0, 0, K])
fig, ax = plt.subplots(2, 2, figsize=(12, 10))
_, _, cbar = dsfilter.visualisations.plot_image_array((U_dilation_anisotropic - U_dilation_isotropic)[..., K], x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[0, 0])
ax[0, 0].set_title(f"$U(\cdot, {θs[0, 0, K]:.2f})$")
fig.colorbar(cbar, ax=ax[0, 0])
_, _, cbar = dsfilter.visualisations.plot_image_array((U_dilation_anisotropic - U_dilation_isotropic).sum(-1), x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[0, 1])
ax[0, 1].set_title("$\int_θ U(\cdot, θ) dθ$")
fig.colorbar(cbar, ax=ax[0, 1])
_, _, cbar = dsfilter.visualisations.plot_image_array((U_erosion_anisotropic - U_erosion_isotropic)[..., K], x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[1, 0])
ax[1, 0].set_title(f"$U(\cdot, {θs[0, 0, K]:.2f})$")
fig.colorbar(cbar, ax=ax[1, 0])
_, _, cbar = dsfilter.visualisations.plot_image_array((U_erosion_anisotropic - U_erosion_isotropic).sum(-1), x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[1, 1])
ax[1, 1].set_title("$\int_θ U(\cdot, θ) dθ$")
fig.colorbar(cbar, ax=ax[1, 1]);

### Shock

In [ ]:
test_case = 1

In [ ]:
match test_case:
    case 1: # Grid of lines (black)
        dim_I, dim_J = 128, 128
        u = np.ones((dim_I, dim_J)) * 255.
        N_lines = 4
        offset = dim_I // (N_lines + 1)
        for k in range(N_lines):
            centre = (k + 1) * offset
            u[(centre-1):(centre+2), :] = 0.
            u[:, (centre-1):(centre+2)] = 0.
        xs, ys = np.meshgrid(np.linspace(-1, 1, dim_I), np.linspace(-1, 1, dim_J))
        mask = (xs**2 + ys**2) < 0.6
        u[mask] = 255.

        G_D_inv = np.array((1., 1., 0.05))

        u = sp.ndimage.gaussian_filter(u, 2)
    case 2: # Grid of lines (alternating black and white)
        dim_I, dim_J = 128, 128
        u = np.ones((dim_I, dim_J)) * 0.5 * 255.
        N_lines = 5
        offset = dim_I // (N_lines + 1)
        colour = 0. # black
        for k in range(N_lines):
            centre = (k + 1) * offset
            u[(centre-1):(centre+2), :] = colour
            u[:, (centre-1):(centre+2)] = 255. - colour
            colour = 255. - colour
        xs, ys = np.meshgrid(np.linspace(-1, 1, dim_I), np.linspace(-1, 1, dim_J))
        mask = (xs**2 + ys**2) < 0.6
        u[mask] = 0.5 * 255.

        G_D_inv = np.array((1., 1., 0.05))

        u = sp.ndimage.gaussian_filter(u, 2)


dim_I, dim_J = u.shape
dim_K = 16
Is, Js, Ks = np.indices((dim_I, dim_J, dim_K))
x_min, x_max = 0., dim_I - 1.
y_min, y_max = 0., dim_J - 1.
θ_min, θ_max = 0., 2 * np.pi
dxy = (x_max - x_min) / (dim_I - 1)
dθ = (θ_max - θ_min) / dim_K
xs, ys, θs = dsfilter.SE2.utils.coordinate_array_to_real(Is, Js, Ks, x_min, y_min, θ_min, dxy, dθ)

In [ ]:
cws = dsfilter.orientationscore.cakewavelet_stack(dim_I, dim_K, Gaussian_σ=dim_I/8)
U = dsfilter.orientationscore.wavelet_transform(u, cws.real)
U = np.transpose(U, axes=(1, 2, 0)) # x, y, θ

In [ ]:
K = 0
print(θs[0, 0, K])
fig, ax = plt.subplots(1, 2, figsize=(12, 5))
_, _, cbar = dsfilter.visualisations.plot_image_array(U[..., K], x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[0])
ax[0].set_title(f"$U(\cdot, {θs[0, 0, K]:.2f})$")
fig.colorbar(cbar, ax=ax[0])
_, _, cbar = dsfilter.visualisations.plot_image_array(U.sum(-1), x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[1])
ax[1].set_title("$\int_θ U(\cdot, θ) dθ$")
fig.colorbar(cbar, ax=ax[1]);

In [ ]:
def shock(U_np, G_inv_np, dxy, dθ, θs_np, T):
    U_dilation = ti.field(dtype=ti.f32, shape=U_np.shape)
    U_dilation.from_numpy(U_np)
    U_erosion = ti.field(dtype=ti.f32, shape=U_np.shape)
    U_erosion.from_numpy(U_np)
    G_inv = ti.Matrix(G_inv_np, dt=ti.f32)
    θs = ti.field(dtype=ti.f32, shape=θs_np.shape)
    θs.from_numpy(θs_np)
    dilation_u = ti.field(dtype=ti.f32, shape=U.shape)
    erosion_u = ti.field(dtype=ti.f32, shape=U.shape)
    τ = compute_timestep_shock(dxy, dθ, G_inv)
    print(τ)
    for _ in tqdm(range(T)):
        morphological(U_dilation, G_inv, dxy, dθ, θs, dilation_u, erosion_u)
        step_dilation(U_dilation, τ, dilation_u)
        step_dilation(U_erosion, τ, erosion_u)
    return U_dilation.to_numpy(), U_erosion.to_numpy()
    
@ti.kernel
def step_dilation(
    U: ti.template(),
    τ: ti.f32,
    dilation_u: ti.template()
):
    for I in ti.grouped(dilation_u):
        U[I] += τ * dilation_u[I]
        
@ti.kernel
def step_erosion(
    U: ti.template(),
    τ: ti.f32,
    erosion_u: ti.template()
):
    for I in ti.grouped(erosion_u):
        U[I] += τ * erosion_u[I]

In [ ]:
U_dilation_isotropic, U_erosion_isotropic = shock(U, np.array((1., 1., 0.001)), dxy, dθ, θs, 50)
U_dilation_anisotropic, U_erosion_anisotropic = shock(U, np.array((1., 0., 0.)), dxy, dθ, θs, 50)

In [ ]:
K = 0
print(θs[0, 0, K])
fig, ax = plt.subplots(2, 2, figsize=(12, 10))
_, _, cbar = dsfilter.visualisations.plot_image_array(U_dilation_anisotropic[..., K], x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[0, 0])
ax[0, 0].set_title(f"$U(\cdot, {θs[0, 0, K]:.2f})$")
fig.colorbar(cbar, ax=ax[0, 0])
_, _, cbar = dsfilter.visualisations.plot_image_array(U_dilation_anisotropic.sum(-1), x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[0, 1])
ax[0, 1].set_title("$\int_θ U(\cdot, θ) dθ$")
fig.colorbar(cbar, ax=ax[0, 1])
_, _, cbar = dsfilter.visualisations.plot_image_array(U_erosion_anisotropic[..., K], x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[1, 0])
ax[1, 0].set_title(f"$U(\cdot, {θs[0, 0, K]:.2f})$")
fig.colorbar(cbar, ax=ax[1, 0])
_, _, cbar = dsfilter.visualisations.plot_image_array(U_erosion_anisotropic.sum(-1), x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[1, 1])
ax[1, 1].set_title("$\int_θ U(\cdot, θ) dθ$")
fig.colorbar(cbar, ax=ax[1, 1]);

In [ ]:
K = 0
print(θs[0, 0, K])
fig, ax = plt.subplots(2, 2, figsize=(12, 10))
_, _, cbar = dsfilter.visualisations.plot_image_array(U_dilation_isotropic[..., K], x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[0, 0])
ax[0, 0].set_title(f"$U(\cdot, {θs[0, 0, K]:.2f})$")
fig.colorbar(cbar, ax=ax[0, 0])
_, _, cbar = dsfilter.visualisations.plot_image_array(U_dilation_isotropic.sum(-1), x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[0, 1])
ax[0, 1].set_title("$\int_θ U(\cdot, θ) dθ$")
fig.colorbar(cbar, ax=ax[0, 1])
_, _, cbar = dsfilter.visualisations.plot_image_array(U_erosion_isotropic[..., K], x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[1, 0])
ax[1, 0].set_title(f"$U(\cdot, {θs[0, 0, K]:.2f})$")
fig.colorbar(cbar, ax=ax[1, 0])
_, _, cbar = dsfilter.visualisations.plot_image_array(U_erosion_isotropic.sum(-1), x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[1, 1])
ax[1, 1].set_title("$\int_θ U(\cdot, θ) dθ$")
fig.colorbar(cbar, ax=ax[1, 1]);

In [ ]:
print(np.max(np.abs(U_dilation_anisotropic - U_dilation_isotropic)))
print(np.max(np.abs(U_erosion_anisotropic - U_erosion_isotropic)))

In [ ]:
K = 0
print(θs[0, 0, K])
fig, ax = plt.subplots(2, 2, figsize=(12, 10))
_, _, cbar = dsfilter.visualisations.plot_image_array((U_dilation_anisotropic - U_dilation_isotropic)[..., K], x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[0, 0])
ax[0, 0].set_title(f"$U(\cdot, {θs[0, 0, K]:.2f})$")
fig.colorbar(cbar, ax=ax[0, 0])
_, _, cbar = dsfilter.visualisations.plot_image_array((U_dilation_anisotropic - U_dilation_isotropic).sum(-1), x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[0, 1])
ax[0, 1].set_title("$\int_θ U(\cdot, θ) dθ$")
fig.colorbar(cbar, ax=ax[0, 1])
_, _, cbar = dsfilter.visualisations.plot_image_array((U_erosion_anisotropic - U_erosion_isotropic)[..., K], x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[1, 0])
ax[1, 0].set_title(f"$U(\cdot, {θs[0, 0, K]:.2f})$")
fig.colorbar(cbar, ax=ax[1, 0])
_, _, cbar = dsfilter.visualisations.plot_image_array((U_erosion_anisotropic - U_erosion_isotropic).sum(-1), x_min, x_max, y_min, y_max, cmap="gray", fig=fig, ax=ax[1, 1])
ax[1, 1].set_title("$\int_θ U(\cdot, θ) dθ$")
fig.colorbar(cbar, ax=ax[1, 1]);

## Diffusion-Shock